<a href="https://colab.research.google.com/github/CRAUGUTH/aiProject/blob/main/Music_Organization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Training NN**


## Installing and Importing Libraries

In [4]:
# Task 1: Install Libraries
!pip install --upgrade spotipy torch pandas scikit-learn transformers librosa gdown

# Task 2: Import Libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np
import joblib
import gdown

## Defining the Model

As described within my progress report 2, I came across many issues when it came to using/implimented pre-trained models on HuggingFace. Because of these probelms I desided to train my own model to classify music based on emotion. My first model was linear, resulting in only a 50% success rate, thus I desided to add multiple layers to the model.

In [5]:
# Task 3: Define Your Model with more layers and dropout
class SongMoodClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SongMoodClassifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.network(x)

## Load and Preprocess Data

Since I had to create my own model to classify music, that means that I also had to find my own data to train the model. This resulting in me searching the web for a dataset which contained the spotify song_id, mood, and the spotify song features. While searching for this I came across a dataset on Kaggle which conatined all this information

Kaggle Link: https://www.kaggle.com/code/muhammadghazimuharam/music-mood-classification

I then downloaded this data to Google Sheets where I then cleaned it up a bit and then uploaded it to my Google Colab Notebook

In [6]:
# Task 4: Load and Preprocess Data
df = pd.read_csv('/content/music_moods.csv')
features = df.drop(['id', 'mood'], axis=1).values
labels = df['mood'].values

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

X_train, X_val, y_train, y_val = train_test_split(features_scaled, labels_encoded, test_size=0.2, random_state=42)

## Train and Validate the Model

Here I am just training the model based on the data from the csv file then I am also validating it to make sure the model is atleast 80% accurate

In [7]:
# Task 5: Train and Validate the Model
input_size = X_train.shape[1]
num_classes = len(np.unique(labels_encoded))

model = SongMoodClassifier(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
batch_size = 65  # You might need to adjust the batch size depending on your dataset

# Function to create batches
def create_batches(X, y, batch_size):
    for i in range(0, len(X), batch_size):
        yield X[i:i+batch_size], y[i:i+batch_size]

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in create_batches(X_train, y_train, batch_size):
        inputs = torch.FloatTensor(X_batch)
        targets = torch.LongTensor(y_batch)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Validation loop
    if epoch % 10 == 0:  # Validate every 10 epochs
        model.eval()
        val_losses = []
        val_accuracies = []
        for X_batch, y_batch in create_batches(X_val, y_val, batch_size):
            inputs = torch.FloatTensor(X_batch)
            targets = torch.LongTensor(y_batch)
            with torch.no_grad():
                outputs = model(inputs)
                val_loss = criterion(outputs, targets)
                val_losses.append(val_loss.item())

                _, predicted = torch.max(outputs, 1)
                val_accuracy = (predicted == targets).sum().item() / targets.size(0)
                val_accuracies.append(val_accuracy)

        avg_val_loss = np.mean(val_losses)
        avg_val_accuracy = np.mean(val_accuracies)
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss}, Validation Accuracy: {avg_val_accuracy}')

Epoch 1/100, Validation Loss: 0.9211404323577881, Validation Accuracy: 0.6551282051282051
Epoch 11/100, Validation Loss: 0.38798391322294873, Validation Accuracy: 0.8256410256410257
Epoch 21/100, Validation Loss: 0.39708952109018963, Validation Accuracy: 0.7788461538461539
Epoch 31/100, Validation Loss: 0.4111839830875397, Validation Accuracy: 0.7891025641025641
Epoch 41/100, Validation Loss: 0.4140947113434474, Validation Accuracy: 0.783974358974359
Epoch 51/100, Validation Loss: 0.4439598123232524, Validation Accuracy: 0.7891025641025641
Epoch 61/100, Validation Loss: 0.43545720477898914, Validation Accuracy: 0.7942307692307692
Epoch 71/100, Validation Loss: 0.44903527200222015, Validation Accuracy: 0.8358974358974359
Epoch 81/100, Validation Loss: 0.4664141039053599, Validation Accuracy: 0.8358974358974359
Epoch 91/100, Validation Loss: 0.5078436533610026, Validation Accuracy: 0.8358974358974359


Here I am just saving the model so that I can use it to classify songs from one of my spotify playlists

In [8]:
# Task 6: Save the Model and Preprocessors
torch.save(model.state_dict(), 'song_mood_classifier.pth')
joblib.dump(label_encoder, 'label_encoder.joblib')
joblib.dump(scaler, 'scaler.joblib')

['scaler.joblib']

# **Music Classification**

## Getting personal playlist for model to classify

Get new access token for Spotify account

In [9]:
client = spotipy.oauth2.SpotifyOAuth(client_id='cce8687d5317494cb81ff7731ccd9a2b',
                                          client_secret='9646e0745dde4bddb7c67b29431a61f0',
                                          redirect_uri='https://example.com/callback/')

sp = spotipy.Spotify(auth_manager=client)

# Get the authorization URL.
authorization_url = client.get_authorize_url()

# Redirect the user to the authorization URL.
print(authorization_url)

https://accounts.spotify.com/authorize?client_id=cce8687d5317494cb81ff7731ccd9a2b&response_type=code&redirect_uri=https%3A%2F%2Fexample.com%2Fcallback%2F


Authorize account

In [10]:
code = "AQD8HOIRWtFUxB0KNXjYfSQ6fgT7ZxFWl6boJYbWoL2BnlU6Q8xT8f9qRiKYD75ZrEQXPzTE7FF7MOxqDSvbe_N5NeHwN3XAs37CjBWGJVHzh_HtcgrdBaY6h_RjoNrHukDZ5Ofvy28KE0AMfke2qCe-t6ewYrP7wW_zOuds-yy7"
client.get_access_token(code)

<ipython-input-10-3b2b8c562b31>:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  client.get_access_token(code)


{'access_token': 'BQAOQmMqmLfMMQqMbUbzYdr7De9ZSiFCUds5mRclTwDCQj4T3xtZ6yWBuM_38X35VG6MyZXT2RZ25iOp8u7JA1IqDCkXBZBvrF2J26Icpcd3w4LqIskToPE6B7wtyWNzj2yrR9fptPy84QQIux5W6EdNGrW9dJ3Yki59qqSRfXFw_3Dg8RlNqEBicv-Vl7uLMeNUCw',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'refresh_token': 'AQBQ_9YLq9y6hPoK1zME9QOl5XPIigaoQmrzdaP08N5lirQWovXf1dzKEZjHn8ibCdMQ4SnOHboZWH3oYYyTcvglGrFNoyF6LVkHhP5AaKWzJU-Kdf8eab9xBiHbSNnVPeE',
 'expires_at': 1699816746,
 'scope': None}

Getting playlist from my account I want to classify

In [11]:
# Get the playlist's ID
playlist_id = '37i9dQZEVXbLp5XoPON0wI'

# Get the playlist's tracks
tracks = sp.playlist_tracks(playlist_id)['items']

# Create a list of song IDs
song_ids = []
for track in tracks:
    song_ids.append(track['track']['id'])

## Classifying each song in playlist

In [12]:
results = {}
features_to_use = [
    'acousticness', 'danceability', 'energy', 'instrumentalness',
    'key', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'
]

for song_id in song_ids:
    # Retrieve song features from Spotify
    song_features = sp.audio_features(song_id)[0]

    # Retrieve song details from Spotify
    track_info = sp.track(song_id)
    song_name = track_info['name']
    artist_name = track_info['artists'][0]['name']  # Assume there is at least one artist

    # Prepare the input tensor for the model
    inputs = [song_features[feature] for feature in features_to_use]
    inputs_tensor = torch.tensor([inputs], dtype=torch.float32)

    # Classify the mood of the song using the model
    with torch.no_grad():
        outputs = model(inputs_tensor)
        _, predicted_class = torch.max(outputs, 1)
        predicted_label = label_encoder.inverse_transform([predicted_class.item()])[0]

    # Store the results including song name, artist, and mood
    results[song_id] = {
        'song_name': song_name,
        'artist_name': artist_name,
        'mood': predicted_label
    }

# Print the classification results along with song name and artist
for song_id, info in results.items():
    print(f"{info['song_name']} by {info['artist_name']}, Mood: {info['mood']}")

Is It Over Now? (Taylor's Version) (From The Vault) by Taylor Swift, Mood: Happy
Now That We Don't Talk (Taylor's Version) (From The Vault) by Taylor Swift, Mood: Calm
I Remember Everything (feat. Kacey Musgraves) by Zach Bryan, Mood: Sad
My Love Mine All Mine by Mitski, Mood: Calm
IDGAF (feat. Yeat) by Drake, Mood: Happy
"Slut!" (Taylor's Version) (From The Vault) by Taylor Swift, Mood: Happy
Paint The Town Red by Doja Cat, Mood: Sad
Stick Season by Noah Kahan, Mood: Happy
Cruel Summer by Taylor Swift, Mood: Sad
Say Don't Go (Taylor's Version) (From The Vault) by Taylor Swift, Mood: Sad
greedy by Tate McRae, Mood: Calm
Agora Hills by Doja Cat, Mood: Calm
MONACO by Bad Bunny, Mood: Sad
fukumean by Gunna, Mood: Sad
500lbs by Lil Tecca, Mood: Sad
Something in the Orange by Zach Bryan, Mood: Calm
First Person Shooter (feat. J. Cole) by Drake, Mood: Sad
Last Night by Morgan Wallen, Mood: Sad
Standing Next to You by Jung Kook, Mood: Happy
Suburban Legends (Taylor's Version) (From The Vault)

# **Playlist Generation**

In [17]:
def display_songs_by_mood(results, mood):
    matching_songs = [(song_id, info) for song_id, info in results.items() if info['mood'].lower() == mood.lower()]

    if not matching_songs:
        print(f"No songs found with the mood: {mood}")
    else:
        print(f"Songs with the mood '{mood}':")
        for song_id, info in matching_songs:
            print(f"{info['song_name']}, by {info['artist_name']}")

In [19]:
# Interactive menu
while True:
    print("\nInteractive Music Mood Search:")
    print("1. Search for Happy songs")
    print("2. Search for Calm songs")
    print("3. Search for Sad songs")
    print("4. Search for Energetic songs")
    print("0. Exit")

    choice = input("Enter your choice (0-4): ")

    if choice == '0':
        print("Exiting the program")
        break
    elif choice == '1':
        display_songs_by_mood(results, 'Happy')
    elif choice == '2':
        display_songs_by_mood(results, 'Calm')
    elif choice == '3':
        display_songs_by_mood(results, 'Sad')
    elif choice == '4':
        display_songs_by_mood(results, 'Energetic')
    else:
        print("Invalid choice. Please enter a number between 0 and 4.")


Interactive Music Mood Search:
1. Search for Happy songs
2. Search for Calm songs
3. Search for Sad songs
4. Search for Energetic songs
0. Exit
Enter your choice (0-4): 1
Songs with the mood 'Happy':
Is It Over Now? (Taylor's Version) (From The Vault), by Taylor Swift
IDGAF (feat. Yeat), by Drake
"Slut!" (Taylor's Version) (From The Vault), by Taylor Swift
Stick Season, by Noah Kahan
Standing Next to You, by Jung Kook
I KNOW ?, by Travis Scott
Thinkin’ Bout Me, by Morgan Wallen
All I Want for Christmas Is You, by Mariah Carey
Ella Baila Sola, by Eslabon Armado
You Proof, by Morgan Wallen

Interactive Music Mood Search:
1. Search for Happy songs
2. Search for Calm songs
3. Search for Sad songs
4. Search for Energetic songs
0. Exit
Enter your choice (0-4): 0
Exiting the program
